In [ ]:
#Download the Python libraries
%pip install pandas
%pip install requests
%pip install plotly

In [ ]:
#import a library
import requests
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
#load from the NASA API

API_KEY = "0SS0PYXqmPsZl3TNmnPeuZamAMRuWDlei826hy0T"
start_date = "2025-09-29"
end_date = "2025-10-04"

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}"

response = requests.get(url)
data = response.json()

In [ ]:
#Converting it to a dataframe"
records = []

for date, neos in data["near_earth_objects"].items():
    for neo in neos:
        diameter = neo["estimated_diameter"]["meters"]
        close_approach = neo["close_approach_data"][0]

        records.append({
            "date": date,
            "id": neo["id"],
            "name": neo["name"],
            "is_hazardous": neo["is_potentially_hazardous_asteroid"],
            "diameter_avg_m": (diameter["estimated_diameter_min"] + diameter["estimated_diameter_max"]) / 2,
            "velocity_km_s": float(close_approach["relative_velocity"]["kilometers_per_second"]),
            "miss_distance_km": float(close_approach["miss_distance"]["kilometers"])
        })
df = pd.DataFrame(records)
df["date"] = pd.to_datetime(df["date"])

In [ ]:
#Creating interactive subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Distribuição dos Diâmetros",
        "Número de NEOs por Dia",
        "Distâncias Mínimas por Data",
        "Velocidade vs Tamanho"
    )
)


In [ ]:
#Histogram of the diameters
fig.add_trace(
    go.Histogram(x=df["diameter_avg_m"], nbinsx=20, marker=dict(color="skyblue")),
    row=1, col=1
)
#Number of NEOs per day
neos_por_dia = df["date"].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=neos_por_dia.index, y=neos_por_dia.values, marker=dict(color="orange")),
    row=1, col=2
)

In [ ]:
#Minimum distance by date
fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["miss_distance_km"]/1000,
        mode="markers", marker=dict(size=8, color="purple"),
        text=df["name"]
    ),
    row=2, col=1
)

In [ ]:
#Velocity vs. Size
fig.add_trace(
    go.Scatter(
        x=df["diameter_avg_m"], y=df["velocity_km_s"],
        mode="markers",
        marker=dict(
            size=10,
            color=df["is_hazardous"].map({True: "red", False: "blue"}),
            opacity=0.7
        ),
        text=df["name"]
    ),
    row=2, col=2
)

In [ ]:
#Overall layout
fig.update_layout(
    height=800, width=1000,
    title_text="Análise de Asteroides Próximos da Terra (NASA NEO Feed)",
    showlegend=False
)

In [ ]:
#Log scale for distances (better visualization)
# Escala log para distâncias (melhor visualização)
fig.update_yaxes(type="log", row=2, col=1, title="Distância mínima (mil km)")
fig.update_xaxes(title="Diâmetro médio (m)", row=1, col=1)
fig.update_xaxes(title="Data", row=1, col=2)
fig.update_xaxes(title="Diâmetro médio (m)", row=2, col=2)
fig.update_yaxes(title="Velocidade (km/s)", row=2, col=2)

fig.show()